In [1]:
import cv2
import numpy as np
import pandas as pd
import sys
import os

from exif import Image
from read_roi import read_roi_file
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm

In [ ]:
IMG_DIR_PATH = "images"
ROI_DIR_PATH = "rois"
FEATURES_FILE_PATH = "tables/features.csv"

In [2]:
table_columns = {'spikelets_num': [],
                 'x_resolution': [],
                 'y_resolution': [],
                 'resolution_unit': [],
                 'compression': [],
                 'exposure_time': [], 
                 'f_number': [],
                 'photographic_sensitivity': [],
                 'subject_distance': [],
                 'focal_length': [],
                 'pixel_x_dimension': [],
                 'pixel_y_dimension': [],
                 'focal_plane_resolution_unit': [], 
                 'exposure_mode': [],
                 'white_balance': [],
                 'name': []}

chosen_meta_features = ['x_resolution',
                        'y_resolution',
                        'resolution_unit',
                        'compression',
                        'exposure_time', 
                        'f_number',
                        'photographic_sensitivity',
                        'subject_distance',
                        'focal_length',
                        'pixel_x_dimension',
                        'pixel_y_dimension',
                        'focal_plane_resolution_unit', 
                        'exposure_mode',
                        'white_balance']

In [3]:
def save_to_file(table):
    table.to_csv(FEATURES_FILE_PATH, index=False)
    
def get_exif_meta(name, ext, feat_dict, row):
    exif_info = Image(IMG_DIR_PATH + "/" + name + ext)
    #all_f = exif_info.list_all()
    #print(all_f)
    for i in range(len(chosen_meta_features)):
        feat = chosen_meta_features[i]
        value = exif_info.get(feat)
        
        row.append(value)
        
        if feat in feat_dict:
            if value in feat_dict[feat]:
                feat_dict[feat][value] += 1
            else:
                feat_dict[feat][value] = 1
        else:
            feat_dict[feat] = {}
            feat_dict[feat][value] = 1

    return feat_dict

In [4]:
def main():
    spikelets = []
    feat_dict = {}
    
    table = pd.DataFrame(table_columns)
    
    for file in tqdm([*(Path(IMG_DIR_PATH)).glob('*.jpg')]):
        name, ext = os.path.splitext(os.path.basename(file))
        roi = read_roi_file(ROI_DIR_PATH + "/" + name + ".roi")
        
        spiklets_num = roi[name]['n']
        spikelets.append(spiklets_num)
        
        img = cv2.imread(IMG_DIR_PATH + "/" + name + ext)
        
        row = []
        row.append(spiklets_num)
        
        feat_dict = get_exif_meta(name, ext, feat_dict, row)
        
        row.append(name)
        
        table.loc[len(table.index)] = row
        
    save_to_file(table)
    
if __name__ == "__main__":
    main()

  0%|          | 0/21 [00:00<?, ?it/s]